### Data Load & Setting

In [28]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel
from keybert import KeyBERT
import ast

In [34]:
data=pd.read_csv('../data/menu_v3_sample_sentence.csv')

# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_name_split"] = data["menu_name_split"].apply(ast.literal_eval)
data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rst_name               40 non-null     object
 1   review_sentence_split  40 non-null     object
 2   menu_name_split        40 non-null     object
 3   org_menu_dict          40 non-null     object
 4   menu_sentence          40 non-null     object
dtypes: object(5)
memory usage: 1.7+ KB


In [36]:
data.head()

,rst_name,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"[, 족발 하 고, 불 족발 새우젓 마늘 쌈장 쌈 야채, 실속 알뜰 보쌈 겉절이, ...","[{' 왕족발+막국수)': ['', '왕족발막국수)', ' 왕족발 막국수)', ' ...",[잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[뼈탕, 양념 주물럭, 모듬한마리, 양념주물럭, 오리 주물럭, 훈제오리, 모듬 한 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 고기시키면 뼈탕이 서비스였는데 들..."
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[토시 살, 된장찌개 밥, 가브리 살, 소고기 세트, 된장찌개밥, 돼지세트 , 김치...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요, 고기도 정말 맛있..."
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","[왕 계 이 란 말 이, 고기반김치찌개세트, 고기 반 김치찌개 세트, 왕계란말이, ...","[{'소불고기': ['소 불고기', '소불고기']}, {'고추장제육': ['고추장제...","[김찌는 맵기조절이가능하여 칼칼하게먹기딱좋고 제육은 불향이나서 짭졸하니 좋아요, 퐁..."
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","[물회, 새꼬막양념, 벌교참꼬막, 간 재미, 새 꼬막, 참 꼬막 양념, 간재미 회무...","[{'쭈꾸미구이': ['쭈꾸미 구이', '쭈꾸미구이']}, {'짱뚱어탕': ['짱뚱...","[벌교참꼬막 38000 2인분 정도 될듯해요, 굴회무침 30 000 간재미무침과 양..."


### Functions

In [7]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith("N") or pos.startswith("SL")):
            results.append(token)
    return results

# 명사 추출 함수
def adverb_remover(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, len_token in result[0][0]:
        if (
            len_token != 1
            and pos.startswith("J") == False
            and pos.startswith("E") == False
            and pos.startswith("MAJ") == False
        ):
            results.append(token)
    return results

# 명사 추출 함수
def extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if (
            len(token) != 1
            and pos.startswith("N")
            or pos.startswith("SL")
            or pos.startswith("V")
        ):
            results.append(token)
    return results

In [ ]:
import ast

# 문자열 형태의 리스트를 파이썬 리스트로 변환
# data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인

### KeyBERT to DataFrame (by. Moonsoo)

In [56]:
# KeyBERT 로드. (KoBERT 사용)

model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

In [54]:
# keybert돌리고 키워드 리턴하는 함수 (!! -- ngram은 무조건 켜야함 -- !!)
def extract_keywords_from_reviews_candy(document,candidate,top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1,3),  # 단어 n-gram 범위
        stop_words=None,  # 불용어F
        # use_maxsum=False,
        # use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        # highlight=True,
        candidates=candidate
    )  # 상위 n개 키워드
    # print(f"Keywords: {keywords}\n")

    return keywords

# 데이터 형식을 input text에 맞게 변환하는 함수 제작 (for문 안에서 돌던거를 위로 뺐음)
def doc_to_input_text(review_doc):
    # 식당의 문장 리스트 추출
    input_text = ".".join(review_doc)
    input_text = adverb_remover(input_text)
    input_text=" ".join(input_text)

    return input_text

In [39]:
# input text 열 만들기
data['input_text'] = data['menu_sentence'].apply(doc_to_input_text)

In [40]:
# 데이터 생긴거 확인 가능

data.head()

,rst_name,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence,input_text
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"[, 족발 하 고, 불 족발 새우젓 마늘 쌈장 쌈 야채, 실속 알뜰 보쌈 겉절이, ...","[{' 왕족발+막국수)': ['', '왕족발막국수)', ' 왕족발 막국수)', ' ...",[잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면...,잡내 맛있 구의 먹자골목 바로 근성 매우 지나가 자주 서비스 메뉴 알차 맵 .. 아...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[뼈탕, 양념 주물럭, 모듬한마리, 양념주물럭, 오리 주물럭, 훈제오리, 모듬 한 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 고기시키면 뼈탕이 서비스였는데 들...",미나리 같이 오리 주물럭 아주 맛있 고기 시키 뼈탕 서비스 들깨 국물 맛있 .. 도...
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[토시 살, 된장찌개 밥, 가브리 살, 소고기 세트, 된장찌개밥, 돼지세트 , 김치...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요, 고기도 정말 맛있...",토시 갈비 섯구이 구성 소고기 세트 주문 고기 정말 맛있 사이드 주문 쫄면 된장찌개...
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","[왕 계 이 란 말 이, 고기반김치찌개세트, 고기 반 김치찌개 세트, 왕계란말이, ...","[{'소불고기': ['소 불고기', '소불고기']}, {'고추장제육': ['고추장제...","[김찌는 맵기조절이가능하여 칼칼하게먹기딱좋고 제육은 불향이나서 짭졸하니 좋아요, 퐁...",김찌 맵기 조절 가능 칼칼하 제육 짭졸 퐁실퐁실 진짜 부드럽 쌉꿀마 김치찌개 계란말...
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","[물회, 새꼬막양념, 벌교참꼬막, 간 재미, 새 꼬막, 참 꼬막 양념, 간재미 회무...","[{'쭈꾸미구이': ['쭈꾸미 구이', '쭈꾸미구이']}, {'짱뚱어탕': ['짱뚱...","[벌교참꼬막 38000 2인분 정도 될듯해요, 굴회무침 30 000 간재미무침과 양...",벌교 꼬막 38000 인분 정도 굴회 무침 30 000 재미 무침 양념 상큼 음식 ...


In [55]:
rst_name_list, menu_name_list, keybert_scores = [], [], []

for i in range(3, 5):
    # 식당의 문장 리스트 추출
    input_text = data.loc[i, 'input_text']
    menu_candidates=data.loc[i, 'menu_name_split']

    rst_name = data.loc[i, 'rst_name']
    results = extract_keywords_from_reviews_candy(input_text,menu_candidates)
    print(results)
    for result in results:
        name, score = result[0], result[1]
    

[('제육', 0.4742), ('간장 제육', 0.3241)]
--------------------------------------------------------------------------------
[('쭈꾸미 구이', 0.7002), ('짱뚱어탕', 0.6895), ('물회', 0.3199)]
--------------------------------------------------------------------------------


### 감성분석 점수

### 데이터프레임 저장